<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---




<p align="center"><h1 align="center">Balling Tabular Classification Tutorial</h1> 

---

<h3 align="center">(Deploy model to an AI Model Share Model Playground REST API<br> and Web Dashboard in five easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/aimstutorialsteps.gif" /></p>


---



## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://aimodelshare.readthedocs.io/en/latest/create_credentials.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*


In [ ]:
#install aimodelshare library
!  pip install aimodelshare --upgrade

In [3]:
# Generate credentials file by entering text inputs
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


## **Set up Environment**

Use your credentials file to set your credentials for all aimodelshare functions. 

In [2]:
# Set credentials 
from aimodelshare.aws import set_credentials

set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [3]:
# Get Balling Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import pickle

In [4]:
#Import the public version of the dataset.
df = pd.read_excel('Balling_Public_5_5.xlsx', engine='openpyxl')
    
#Define features
feat = ['Solid/Spread','LED [J/m]','Dwell Time [s]']

#Create feature array
X = df.loc[:,feat]
y = df['Balling']



In [5]:
#Create training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=31)


## **(1) Preprocessor Function & Setup**

### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For tabular prediction models users should minimally include function inputs for an unpreprocessed pandas dataframe.*  
*  *Any categorical features should be preprocessed to one hot encoded numeric values.* 


-- Here is where we actually write the preprocessor function:


In [6]:
# Write function to transform data with preprocessor 
# In this case we use panda's fillna in our preprocessor function to replace missing values with column means

def preprocessor(data):
    preprocessed_data=data.fillna({'Solid/Spread': 26.428271, 'LED [J/m]': 233.049568, 'Dwell Time [s]': 0.000125})

    return preprocessed_data.values #return preprocessed numpy array

In [7]:
# check shape of X data after preprocessing it using our new function
preprocessor(X_train)

array([[2.49274536e+00, 3.33333333e+02, 1.66666667e-04],
       [3.69828987e+00, 2.68421053e+02, 1.05263158e-04],
       [3.79560823e+00, 6.10389610e+01, 5.19480519e-05],
       ...,
       [1.61384108e+01, 1.96191575e+02, 5.77034045e-05],
       [3.32264299e+00, 4.62500000e+02, 1.25000000e-04],
       [2.19114737e+01, 4.00000000e+02, 1.25000000e-04]])

## **(2) Build Model Using sklearn (or Your Preferred ML Library)**

### **Logistic Regression with L1 Regularization (Lasso)**

In [8]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, penalty='l1', solver = 'liblinear')
model.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model.score(preprocessor(X_train), y_train) # Fit score, 0-1 scale.

0.874933190807055

## **(3) Save Preprocessor**
### Saves preprocessor function to "preprocessor.zip" file

In [9]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [10]:
#  Now let's import and test the preprocessor function to see if it is working...

import aimodelshare as ai
prep=ai.import_preprocessor("preprocessor.zip")

prep(X_test).shape

(624, 3)

## **(4) Save sklearn model to Onnx File Format**


In [11]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # You need to insert correct number of preprocessed features

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## **(5) Create your Model Playground and Deploy REST API/ Live Web-Application**

In [12]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata = X_train.head()
email_list=["brentvela@tamu.edu","rarroyave@tamu.edu","sina@tamu.edu","sofiasheikh@exchange.tamu.edu","mp3675@columbia.edu"]

In [13]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class

myplayground=ModelPlayground(model_type="tabular", classification=True, private=True, email_list=email_list)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, y_train, exampledata) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):Balling Classification Model Playground
Model Description (Explain what your model does and 
 why end-users would find your model useful):Predict balling from alloy chemistry.
Model Key Words (Search categories that describe your model, separated with commas):balling, alloy, chemistry, classification
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 80 seconds. 
 Playground Url: "https://ov8kxwsss6.execute-api.us-east-2.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic tab.
https://www.mo

## **Use your new Model Playground!**

Follow the link in the output above to:
- Generate predictions with your interactive web dashboard
- Access example code in Python, R, and Curl

Or, follow the rest of the tutorial to create a competition for your Model Playground and: 
- Access verified model performance metrics 
- Upload multiple models to a leaderboard 
- Easily compare model performance & structure 

## **Part 2: Create a Competition**

-------

After deploying your Model Playground, you can now create a competition. 

Creating a competition allows you to:
1. Verify the model performance metrics on aimodelshare.org
2. Submit models to a leaderboard
3. Grant access to other users to submit models to the leaderboard
4. Easily compare model performance and structure 

In [14]:
# Create list of authorized participants for competition
# Note that participants should use the same email address when creating modelshare.org account

email_list=["brentvela@tamu.edu","rarroyave@tamu.edu","sina@tamu.edu","sofiasheikh@exchange.tamu.edu","mp3675@columbia.edu"]

In [15]:
#Save X_train, y_train, and X_test to a directory that stores competition data
import os

#make folder for competition data
os.mkdir("balling_competition_data")

#save data to folder
X_train.to_csv("balling_competition_data/X_train.csv",index=False)
y_train.to_csv("balling_competition_data/y_train.csv",index=False)
X_test.to_csv("balling_competition_data/X_test.csv",index=False)

In [16]:
# Create Competition
myplayground.create_competition(data_directory='balling_competition_data', 
                                y_test = y_test, 
                                email_list=email_list)


--INPUT COMPETITION DETAILS--

Enter competition name:Alloy Balling Competition
Enter competition description:Classify balling from alloy chemistry features.

--INPUT DATA DETAILS--

Note: (optional) Save an optional LICENSE.txt file in your competition data directory to make users aware of any restrictions on data sharing/usage.

Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Data stored in balling competition folder.
Enter optional data license descriptive name (e.g.- 'MIT, Apache 2.0, CC0, Other, etc.'):MIT
Uploading your data. Please wait for a confirmation message.

 Success! Model competition created. 

You may now update your prediction API runtime model and verify evaluation metrics with the update_runtime_model() function.

To upload new models and/or preprocessors to this API, team members should use 
the following credentials:

apiurl='https://ov8kxwsss6.execute-api.us-east-2.amazonaws.com/prod

In [17]:
#Instantiate Competition
#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://2121212.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

playground_id='https://ov8kxwsss6.execute-api.us-east-2.amazonaws.com/prod/m'

mycompetition= ai.Competition(playground_id)

In [ ]:
# Add, remove, or completely update authorized participants for competition later
emaillist=["emailaddress4@email.com"]

mycompetition.update_access_list(email_list=emaillist,update_type="Add")

Submit Models

In [18]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

set_credentials(apiurl=playground_id) # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m



AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [19]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted labels "survived" or "died") (Model 1)
prediction_labels = model.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2712


In [28]:
# Create model 2 (L2 Regularization - Ridge)
from sklearn.linear_model import LogisticRegression

model_2 = LogisticRegression(C=.01, penalty='l2',class_weight="balanced")
model_2.fit(preprocessor(X_train), y_train) # Fitting to the training set.
model_2.score(preprocessor(X_train), y_train) # Fit score, 0-1 scale.

0.51309460181721

In [29]:
# Save Model 2 to .onnx file

# How many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # You need to insert correct number of preprocessed features

onnx_model = model_to_onnx(model_2, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString()) 

In [30]:
# Submit Model 2

#-- Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 2

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2712


Get Leaderboard

In [31]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,num_params,optimizer,username,version
0,85.90%,46.21%,42.95%,50.00%,sklearn,False,False,LogisticRegression,3,liblinear,mikedparrott,1
1,49.04%,44.95%,55.41%,60.84%,sklearn,False,False,LogisticRegression,3,lbfgs,mikedparrott,2


Compare Models

In [32]:
# Compare two or more models (Experimental, Git-like Diffs for Model Architectures)
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)


,param_name,default_value,model_version_1,model_version_2
0,C,1.000000,10,0.010000
1,class_weight,None,None,balanced
2,dual,False,False,False
3,fit_intercept,True,True,True
4,intercept_scaling,1,1,1
5,l1_ratio,None,None,None
6,max_iter,100,100,100
7,multi_class,auto,auto,auto
8,n_jobs,None,None,None
9,penalty,l2,l1,l2


#### Check structure of y test data 
(This helps users understand how to submit predicted values to leaderboard)

In [33]:
mycompetition.inspect_y_test()

{'ytest_example': [0, 0, 0, 0, 0],
 'y_length': 624,
 'class_labels': [0, 1],
 'class_balance': {'0': 537, '1': 87},
 'label_dtypes': {"<class 'int'>": 624}}

## **Part 3: Maintaining your Model Playground**

-------

Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the modelperformance metrics in your Model Playground*

In [ ]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class

myplayground=ModelPlayground(playground_url=playground_id)

myplayground.update_runtime_model(model_version=2)

Delete Deployment 

*Use this function to delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models*

In [ ]:
myplayground.delete_deployment()